# Download the source file

In [0]:
!wget https://raw.githubusercontent.com/nmrksic/neural-belief-tracker/master/data/woz/woz_train_en.json
!touch formatted_file.jsonl

In [0]:
import json

def format_file():
  with open("formatted_file.jsonl", 'w+') as split_file:
    with open ("woz_train_en.json") as src_file:
      dialogues = json.loads(src_file.read())
      for di, d in enumerate(dialogues):
        previous_state = {'inform': [], 'request': []}
        turns = d['dialogue']
        for ti, t in enumerate(turns):
            question = 'What is the change in state?'
            actions = []
            for act in t['system_acts']:
                if isinstance(act, list):
                    act = ': '.join(act)
                actions.append(act)
            actions = ', '.join(actions)
            if len(actions) > 0:
                actions += ' -- '
            context = actions + t['transcript']
            belief_state = t['belief_state']
            delta_state = {'inform': [], 'request': []}
            current_state = {'inform': [], 'request': []}
            for item in belief_state:
                if 'slots' in item:
                    slots = item['slots']
                    for slot in slots:
                        act = item['act']
                        if act == 'inform':
                            current_state['inform'].append(slot)
                            if not slot in previous_state['inform']:
                                delta_state['inform'].append(slot)
                            else:
                                prev_slot = previous_state['inform'][previous_state['inform'].index(slot)]
                                if prev_slot[1] != slot[1]:
                                    delta_state['inform'].append(slot)
                        else: 
                            delta_state['request'].append(slot[1])
                            current_state['request'].append(slot[1])
            previous_state = current_state
            answer = ''
            if len(delta_state['inform']) > 0:
                answer = ', '.join([f'{x[0]}: {x[1]}' for x in delta_state['inform']])
            answer += ';'
            if len(delta_state['request']) > 0:
                answer += ' '
                answer += ', '.join(delta_state['request'])
            ex = {'context': ' '.join(context.split()), 
                 'question': ' '.join(question.split()),
                 'answer': answer if len(answer) > 1 else 'None'
                 }
            split_file.write(json.dumps(ex)+'\n')

format_file()       